<html>
<div style="background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 600px; vertical-align: middle; height: 40px; margin: 10px;">
<h1 style="font-family: Georgia; color: black;">AI-Fall 01-CA2-Part1</h1>
</div>
<div style="background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 500px; margin: 10px;">
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/f/fd/University_of_Tehran_logo.svg/225px-University_of_Tehran_logo.svg.png" width=60px width=auto style="padding:10px; vertical-align: middle;">
  <span style="font-family: Georgia; font-size:30px; color: black;">University of Tehran </span>
</div>
<div style=" background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 400px; height: 30px; margin: 10px;">
  <span style="font-family: Georgia; font-size:15pt; color: black; vertical-align: middle;">Saman Eslami Nazari - std id: 810199375 </span>
</div>
</html>

First I need a function to evaluate math expressions given in strings. For this purpose I will used the code below, which is taken from [this answer on Stack Overflow](https://stackoverflow.com/a/9558001/14839317).

In [1]:
import ast
import operator as op

# supported operators
operators = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
             ast.Div: op.truediv, ast.Pow: op.pow, ast.BitXor: op.xor,
             ast.USub: op.neg, ast.Mod: op.mod}

def eval_expr(expr):
    return eval_(ast.parse(expr, mode='eval').body)

def eval_(node):
    if isinstance(node, ast.Num): # <number>
        return node.n
    elif isinstance(node, ast.BinOp): # <left> <operator> <right>
        return operators[type(node.op)](eval_(node.left), eval_(node.right))
    elif isinstance(node, ast.UnaryOp): # <operator> <operand> e.g., -1
        return operators[type(node.op)](eval_(node.operand))
    else:
        raise TypeError(node)

# Part 1: Clarifying basic concepts

First of all we need to define our genes and chromosomes. The genes will be the given numbers and operators as different permutations of both can lead to different answers. The chromosomes will be an answers to the problem, therefore they should be a valid permutation of the genes (numbers and operators). Chromosomes will be numbers with one operator placed between every two of them. I will make sure that this constraint will be held true during crossovers and other parts of the algorithm.
No lets create the `Chromosome` class.

In [2]:
import math


class Chromosome:
    def __init__(self, expr_list: list[str]):
        self._operands = expr_list[0::2]
        self._operators = expr_list[1::2]
        self._expr_list = expr_list
        self._expr = ''.join(expr_list)
        self._len = len(expr_list)

    def to_list(self) -> list[str]:
        return self._expr_list

    def change_gene(self, position: int, new_val: str) -> None:
        self._expr_list[position] = new_val
        self._operands = self._expr_list[0::2]
        self._operators = self._expr_list[1::2]
        self._expr = ''.join(self._expr_list)

    def __str__(self) -> str:
        return self._expr

    def __len__(self) -> int:
        return self._len

    def __eq__(self, __o: object) -> bool:
        return self._expr == __o._expr


# Initialize population:

To initialize the chromosomes, I will create a function that gets the population size and the given operands and operators as input and returns a list of chromosomes.

In [3]:
import random


def init_population(
        population_size: int,
        chrom_len: int,
        operators: list[str],
        operands: list[str]) -> list[Chromosome]:

    population: list[Chromosome] = []
    for _ in range(population_size):
        current_chromo_list = [None] * chrom_len
        current_chromo_list[0::2] =  \
            random.choices(operands, k=math.ceil(chrom_len / 2))
        current_chromo_list[1::2] = \
            random.choices(operators, k=math.floor(chrom_len / 2))
        population.append(Chromosome(current_chromo_list))
    return population


# Implementation and fitness function:

I will use the difference between the calculated value of the left hand side and the right hand side of the equation. The lesser this value is, the better the chromosome will be. In order to have greater score for better chromosomes, I will use the following function to transform this value to the fitness score:
$$diff = |lhs(equation) - rhs(equation)|$$
$$Fitness(diff) = \frac{1}{1+diff}$$
I used $1 + diff$ instead of $diff$ to handle the case of $diff = 0$.

In [4]:
from itertools import accumulate
from bisect import bisect_left
from copy import deepcopy


def fitness_score(chrom: Chromosome, goal: int) -> int:
    diff = abs(int(eval_expr(str(chrom))) - goal)
    return 1 / (1 + diff)


def select_mating_pool(
        chroms: list[Chromosome],
        fitness_score_list: list[int]) -> list[Chromosome]:

    fitness_sum: int = sum(fitness_score_list)
    prob_list: list[float] = [f/fitness_sum for f in fitness_score_list]
    cum_prob_list: list[float] = list(accumulate(prob_list))
    
    mating_pool: list[Chromosome] = []
    for _ in range(len(chroms)):
        rand_point = random.uniform(0, 1)
        chosen_chromo = bisect_left(cum_prob_list, rand_point)
        mating_pool.append(deepcopy(chroms[chosen_chromo]))
    return mating_pool


# Crossover, mutation and creating next generation

First, lets define a `crossover` function that takes parents as input and returns the offspring.

In [5]:
def crossover(parent1: Chromosome, parent2: Chromosome) -> Chromosome:
    child = parent1.to_list()
    parent2_list = parent2.to_list()
    cut_index = random.randint(0, len(child))
    child[:cut_index], parent2_list[:cut_index] = parent2_list[:cut_index], child[:cut_index]
    return Chromosome(child)


After that, I need to define a `mutate` function that takes a chromosome and returns the new mutated chromosome.

In [6]:
def mutate(
        chrom: Chromosome,
        mutation_rate: float,
        operators: list[str],
        operands: list[str]) -> None:

    for i in range(len(chrom)):
        if random.uniform(0, 1) > mutation_rate:
            continue

        new_gene: str
        if i % 2 == 1:  # is operator
            new_gene = random.choice(operators)
        else:  # is operand
            new_gene = random.choice(operands)
        chrom.change_gene(i, new_gene)


Now, I will create the function `create_next_generation` that will use the previous functions to make the next generation of chromosomes.

In [7]:
def create_next_generation(
        mating_pool: list[Chromosome],
        cross_over_rate: float,
        mutation_rate: float,
        operators: list[str],
        operands: list[str]) -> list[Chromosome]:

    next_gen: list[Chromosome] = deepcopy(mating_pool)
    for i in range(len(mating_pool)):
        if random.uniform(0, 1) > cross_over_rate:
            continue

        next_gen[i] = crossover(
            mating_pool[i], mating_pool[(i + 1) % len(mating_pool)]
        )
    for i in range(len(next_gen)):
        mutate(next_gen[i], mutation_rate, operators, operands)

    return next_gen


# Implementing the Genetic Algorithm on the problem

In the end, I will gather these functions together and call them to solve the problem. First, I will create the data class `ProblemInfo` to hold the problem's information.

In [8]:
from dataclasses import dataclass


@dataclass
class ProblemInfo:
    goal: int
    operands: list[str]
    operators: list[str]
    eq_len: int


I will also need a `HyperParams` class for storing values of initial population size, cross over rate and mutation rate.

In [9]:
@dataclass
class HyperParams:
    init_population_size: int
    cross_over_rate: float
    mutation_rate: float
    max_steps: int


Problem solving implementation:

In [10]:
def g_alg_solve(problem_info: ProblemInfo, hyper_params: HyperParams) -> str:
    problem_info = deepcopy(problem_info)
    hyper_params = deepcopy(hyper_params)

    population: list[Chromosome] = init_population(
        hyper_params.init_population_size,
        problem_info.eq_len,
        problem_info.operators,
        problem_info.operands
    )

    while hyper_params.max_steps != 0:
        fitness_score_list = [
            fitness_score(c, problem_info.goal) for c in population
        ]

        if 1 in fitness_score_list:
            return str(population[fitness_score_list.index(1)])

        mating_pool = select_mating_pool(population, fitness_score_list)
        population = create_next_generation(
            mating_pool,
            hyper_params.cross_over_rate,
            hyper_params.mutation_rate,
            problem_info.operators,
            problem_info.operands
        )
        hyper_params.max_steps -= 1


Testing the problem solver:

In [12]:
problem_info = ProblemInfo(
    goal=-123456,
    operands=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
    operators=['+', '-', '*'],
    eq_len=25
)

hyper_params = HyperParams(
    init_population_size=100,
    cross_over_rate=0.8,
    mutation_rate=0.1,
    max_steps=-1
)

print(f"answer : {g_alg_solve(problem_info, hyper_params)}")


answer : 10-10*8*8*10*10*2-2+2*7*9*4*9
